In [1]:
using QuantumCollocation
using NamedTrajectories
using TrajectoryIndexingUtils
using Interpolations
using JLD2

Precompiling QuantumCollocation
  ✓ QuantumCollocation
  1 dependency successfully precompiled in 97 seconds. 482 already precompiled.


In [2]:
const Units = 1e9
const MHz = 1e6 / Units
const GHz = 1e9 / Units
const ns = 1e-9 * Units
const μs = 1e-6 * Units
;

In [3]:
# Operators
const Paulis = Dict(
    "I" => Matrix{ComplexF64}([1 0; 0 1]),
    "X" => Matrix{ComplexF64}([0 1; 1 0]),
    "Y" => Matrix{ComplexF64}([0 im; -im 0]),
    "Z" => Matrix{ComplexF64}([1 0; 0 -1]),
)
const a = [0 1; 0 0]
const ad = transpose(a);
rz_op(theta) = exp(-im/2 * theta[1] * Paulis["Z"]);

RZ = Gate(1,rz_op)

u2_op(theta) = 1/sqrt(2) * [1 -exp(1im * theta[2]);exp(1im * theta[1]) exp(1im * (theta[1]+theta[2]))]


u2_op (generic function with 1 method)

In [4]:
H_drives = [
     Paulis["X"],Paulis["Y"]
]
system = QuantumSystem(H_drives);
t_f = 10* ns
n_steps = 51
times = range(0, t_f, n_steps)  # Alternative: collect(0:Δt:t_f)
n_controls=1
n_qubits=1;
Δt = times[2] - times[1]

0.2

In [5]:
N = 11
### Generate Initial Trajectories 
PICO_max_iter = 100

# Shape the cost function with weights on states and controls
Q = 100.
R =1e-2

a_bound = 1.0
dda_bound = 1.0

ops = IpoptOptions()
ops.print_info_string = "yes"
ops.recalc_y = "yes"
ops.recalc_y_feas_tol = 1.0 ##down
ops.print_level = 0

ops2=deepcopy(ops)
ops2.print_level = 5

p_op = PiccoloOptions()
p_op.verbose=false
p_op.pade_order=10
p_op.free_time=false
p_op.timesteps_all_equal=true

true

In [6]:
RZ_dda_trajectory = pretrain(
    RZ,
    11,
    system,
    n_steps,
    Δt,
    PICO_max_iter,
    1-1e-5;
    Q_symb=:dda,
    Q=Q,
    R=R,
    a_bound=a_bound,
    dda_bound=dda_bound,
    ipopt_options1=ops,
    ipopt_options2=ops2,
    piccolo_options=p_op
);


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:    41250
Number of nonzeros in inequality constraint Jacobian.:       44
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:     7656
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     2178
                     variables with only upper bounds:        0
Total number of equality constraints.................:     6600
Total number of inequality c

In [7]:
RZ_U_trajectory = pretrain(
    RZ,
    11,
    system,
    n_steps,
    Δt,
    PICO_max_iter,
    1-1e-5;
    Q_symb=:Ũ⃗,
    Q=Q,
    R=R,
    a_bound=a_bound,
    dda_bound=dda_bound,
    ipopt_options1=ops,
    ipopt_options2=ops2,
    piccolo_options=p_op
);

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:    41250
Number of nonzeros in inequality constraint Jacobian.:       44
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:     7656
                     variables with only lower bounds:        0
                variables with lower and upper bounds:     2178
                     variables with only upper bounds:        0
Total number of equality constraints.................:     6600
Total number of inequality constraints...............:       11
        inequality constraints with only lower bounds:       11
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  6.7481542e+01 1.87e+00 3.97e-01   0.0 0.00e+00    -  0.00e+00 0.00e+00 

In [8]:
JLD2.save("RZ_dda_trajectory.jld2",RZ_dda_trajectory)
JLD2.save("RZ_U_trajectory.jld2",RZ_U_trajectory)

In [8]:
U2_trajectory = pretrain(
    U2,
    11,
    system,
    n_steps,
    Δt,
    PICO_max_iter,
    1-1e-5;
    Q=Q,
    R=R,
    a_bound=a_bound,
    dda_bound=dda_bound,
    ipopt_options1=ops,
    ipopt_options2=ops2,
    piccolo_options=p_op
);

This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:   453750
Number of nonzeros in inequality constraint Jacobian.:      968
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:    84216
                     variables with only lower bounds:        0
                variables with lower and upper bounds:    23958
                     variables with only upper bounds:        0
Total number of equality constraints.................:    72600
Total number of inequality constraints...............:      121
        inequality constraints with only lower bounds:      121
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  2.8291206e+00 1.88e+00 3.04e-01   0.0 0.00e+00    -  0.00e+00 0.00e+00 

In [ ]:
U2_trajectory = pretrain(
    U2,
    11,
    system,
    n_steps,
    Δt,
    PICO_max_iter,
    1-1e-5;
    Q=Q,
    R=R,
    a_bound=a_bound,
    dda_bound=dda_bound,
    ipopt_options1=ops,
    ipopt_options2=ops2,
    piccolo_options=p_op
);

In [9]:
JLD2.save("RZ_pretrained.jld2",RZ_trajectory)

In [10]:
JLD2.save("U2_pretrained.jld2",U2_trajectory)